In [1]:
!pip install -q internetarchive


In [2]:
import internetarchive
import os
import tarfile
import csv
from datetime import datetime
from tqdm.notebook import tqdm
import pandas as pd
import configparser
import json
from zipfile import ZipFile

import warnings

# Suppressing the warnings
warnings.filterwarnings('ignore') 
     

In [3]:
path_d = r'C:\Users\35383\Data Analytics - Jupyter\AgoraVai\downloads' 


if not os.path.exists(path_d):
    os.makedirs(path_d)
    print("{} created.".format(path_d))

In [4]:
i = 'archiveteam-twitter-stream-2021-01'

file_names = [f.name for f in internetarchive.get_files(i, glob_pattern= '*')]
file_names

['__ia_thumb.jpg',
 'archiveteam-twitter-stream-2021-01_archive.torrent',
 'archiveteam-twitter-stream-2021-01_files.xml',
 'archiveteam-twitter-stream-2021-01_meta.sqlite',
 'archiveteam-twitter-stream-2021-01_meta.xml',
 'archiveteam-twitter-stream-2021-01_reviews.xml',
 'twitter-stream-2021-01-01.zip',
 'twitter-stream-2021-01-02.zip',
 'twitter-stream-2021-01-03.zip',
 'twitter-stream-2021-01-04.zip',
 'twitter-stream-2021-01-05.zip',
 'twitter-stream-2021-01-06.zip',
 'twitter-stream-2021-01-25.zip',
 'twitter-stream-2021-01-26.zip',
 'twitter-stream-2021-01-27.zip',
 'twitter-stream-2021-01-28.zip',
 'twitter-stream-2021-01-29.zip',
 'twitter-stream-2021-01-30.zip',
 'twitter-stream-2021-01-31.zip',
 'twitterlogo.png',
 'twitterlogo_thumb.jpg']

In [5]:
def donwload(file, i = i, path_d = path_d):
    
    item =  internetarchive.get_item(i)
    
    #downloading
    r = item.download(
      destdir=path_d,  # The directory to download files to
      ignore_existing=True,  # Skip files that already exist locally
      checksum=True,  # Skip files based on checksum
      verbose=True,  # Print progress to stdout
      retries=100,  # The number of times to retry on failed requests
      no_directory=True,  # Download withtout the identifier
      files = file)
    
    path_file = path_d + f'/{file}'
    
    return path_file 

In [6]:
keywords = ['civil service', 'civil service employment', 'civil service salary', 'civil service earnings',
                  'defence', 'defence employment', 'defence salary', 'defence earnings',
                  'educators', 'availability of educators', 'educators salary', 'educator earnings',
                  'garda siochona', 'garda salary', 'garda earnings', 'garda of numbers',
                  'health employment numbers', 'health salary', 'health earnings',
                  'public service', 'public sector'] 

In [24]:
f = 'twitter-stream-2021-01-06.zip'

archive = donwload(f)

archiveteam-twitter-stream-2021-01:
 downloading twitter-stream-2021-01-06.zip: 100%|██████████████████████████████████| 970M/970M [09:42<00:00, 1.74MiB/s]


In [15]:
def get_tweets(archive):

    with ZipFile(archive, 'r') as zip:

        zip_file_names = list(filter(lambda t: t.endswith('.json.bz2'), zip.namelist()))
        zip_file_names


        for zfn in tqdm(zip_file_names, desc='Progress:'):

            zip.extract(zfn,path=path_d+"/json")

            js_path = path_d+"/json/"+zfn

            js_path = js_path.replace("/", "\\")
            #print(js_path)

            #reading json
            df = pd.read_json(js_path, lines=True, compression='bz2')

            df = df[df.lang == 'en']

            bow = '|'.join(keywords) # bag of word
            df = df[df['text'].str.contains(bow, case=False)]

            if zfn == zip_file_names[0]:
                tweets = df

            else:
                tweets = tweets.append(df)


            os.remove(js_path)

        pass
    
    return tweets

In [25]:
df_6 = get_tweets(archive)

Progress::   0%|          | 0/546 [00:00<?, ?it/s]

In [26]:
tweets = pd.read_csv(path_d+'/tweets.csv')
tweets = tweets.append(df_6)
tweets.to_csv(path_d+'/tweets.csv', index=False)

In [27]:
tweets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3373 entries, 0 to 1000
Data columns (total 39 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   created_at                 3373 non-null   object 
 1   id                         3373 non-null   float64
 2   id_str                     3373 non-null   float64
 3   text                       3373 non-null   object 
 4   source                     3373 non-null   object 
 5   truncated                  3373 non-null   float64
 6   in_reply_to_status_id      982 non-null    float64
 7   in_reply_to_status_id_str  982 non-null    float64
 8   in_reply_to_user_id        1002 non-null   float64
 9   in_reply_to_user_id_str    1002 non-null   float64
 10  in_reply_to_screen_name    1002 non-null   object 
 11  user                       3373 non-null   object 
 12  geo                        2 non-null      object 
 13  coordinates                2 non-null      objec

In [17]:
tweets.isnull().sum()

created_at                      0
id                              0
id_str                          0
text                            0
source                          0
truncated                       0
in_reply_to_status_id        1395
in_reply_to_status_id_str    1395
in_reply_to_user_id          1385
in_reply_to_user_id_str      1385
in_reply_to_screen_name      1385
user                            0
geo                          1990
coordinates                  1990
place                        1976
contributors                 1991
is_quote_status                 0
quote_count                     0
reply_count                     0
retweet_count                   0
favorite_count                  0
entities                        0
favorited                       0
retweeted                       0
filter_level                    0
lang                            0
timestamp_ms                    0
display_text_range           1388
retweeted_status              810
possibly_sensi